In [ ]:
from fastapi import FastAPI
# FastAPI: Criar e gerenciar a API.

In [ ]:
from pydantic import BaseModel
# pydantic.BaseModel: Validar dados recebidos em requisições.

In [5]:
import pandas as pd
# pandas: Manipular os dados da base.

In [6]:
import numpy as np
# numpy: Cálculos numéricos.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity: Medir semelhança entre usuários com base nas avaliações.

In [9]:
from sklearn.impute import SimpleImputer
# SimpleImputer: Preencher valores ausentes (NaN) com a média.

In [11]:
df = pd.read_csv("base_recomendacao.csv")
# Lê um arquivo CSV com as colunas user_id, product_id e rating.

In [12]:
user_item_matrix = df.pivot_table(index='user_id', columns='product_id', values='rating')
#ransforma esses dados em uma matriz de utilidade onde:

#Linhas: usuários
#Colunas: produtos
#Valores: avaliações dadas (ratings)

In [13]:
imputer = SimpleImputer(strategy='mean')
#Preenche os valores NaN com a média das avaliações (por produto).

In [14]:
user_item_matrix_filled = imputer.fit_transform(user_item_matrix)
# cria uma nova matriz com todos os dados preenchidos.

In [15]:
user_similarity = cosine_similarity(user_item_matrix_filled)
# Calcula a similaridade do cosseno entre cada par de usuários.
# Retorna uma matriz user x user, onde cada célula indica o quão parecidos são dois usuários com base nas avaliações.


In [16]:
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
# Transforma essa matriz em um DataFrame para facilitar o acesso.

In [22]:
def recommend_products(user_id, user_item_matrix, similarity_matrix, top_n=3):
#Inicia a definição da função chamada recommend_products.
#parâmetros: 
#user_id(int) - O ID do usuário para o qual você deseja gerar recomendações.
#user_item_matrix(DataFrame) - A matriz usuário x produto com os ratings (alguns podem ser NaN).
#similarity_matrix (np.array (2D)) - A matriz de similaridade entre os usuários (ex: cosseno).
#top_n(int,padrão=3) - Quantidade de produtos que você deseja recomendar
    user_idx = user_item_matrix.index.get_loc(user_id)
    #Pega a posição do usuário na matriz.
    similar_users = similarity_matrix[user_idx]
    #Vetor de similaridade desse usuário com os outros.
    scores = np.dot(similar_users, user_item_matrix_filled)
    # Produto escalar da similaridade com a matriz de ratings preenchida → estima o quanto ele "gostaria" dos produtos.
    user_rated = user_item_matrix.loc[user_id].dropna().index
    #Pega os produtos que o usuário já avaliou (não queremos recomendar isso).
    recommendations = pd.Series(scores, index=user_item_matrix.columns).drop(user_rated, errors="ignore")
    #Remove os produtos já avaliados e ordena os demais.
    top_recommendations = recommendations.sort_values(ascending=False).head(top_n)
    #Pega os top_n com maior "score".
    return top_recommendations.index.tolist()
    #retornar a lista final de recomendações.

In [23]:
app = FastAPI()
#app = FastAPI()

In [26]:
class RecommenderRequest(BaseModel):
    #Define a estrutura dos dados esperados na requisição:
    user_id: int
    #usuário para o qual será feita a recomendação.
    top_n: int = 3
    #top_n: quantidade de produtos recomendados (padrão = 3).


In [28]:
@app.post("/recomendar")
#Define a rota POST /recomendar.
def recomendar(request: RecommenderRequest):
    #Recebe o corpo da requisição como um objeto do tipo RecommenderRequest.
    recs = recommend_products(request.user_id, user_item_matrix, user_similarity_df.values, request.top_n)
    #Chama a função de recomendação.
    return {"recomendados": recs}
    #Retorna um JSON com os produtos recomendados.
